In [42]:
import os

import pandas as pd
import numpy as np
import scipy.signal as signal

import matplotlib.pyplot as plt
from matplotlib import rcParams

In [43]:
config = {
    "font.family": 'Times New Roman',
    "axes.unicode_minus": False,
    "font.size": 12
}
rcParams.update(config)

In [44]:
sheet = pd.read_excel('data')
data = sheet.astype('float32').T

In [45]:
time = data.iloc[0]
dt = time[1] - time[0]
n = len(time)

In [46]:
freq = np.fft.fftfreq(n, d=dt)  # 计算频率轴

def calculate(row):
    # 傅里叶变换
    fft_result = np.fft.fft(row)
    positive_freq_indices = np.where(freq >= 0)
    positive_fft_result = fft_result[positive_freq_indices]


    # 傅里叶变换结果的幅值谱
    amplitude_spectrum = np.abs(fft_result)
    positive_amplitude_spectrum = np.abs(positive_fft_result)
    # 检索幅值谱峰值
    peaks, _ = signal.find_peaks(positive_amplitude_spectrum)
    # 获取最大峰值的索引
    max_peak_index = np.argmax(positive_amplitude_spectrum[peaks])

    # 获取最大峰值对应的频率和幅值
    resonant_frequency = freq[peaks[max_peak_index]]
    max_peak_amplitude = positive_amplitude_spectrum[peaks[max_peak_index]]
    
    return resonant_frequency, max_peak_amplitude, amplitude_spectrum

In [47]:
dict = {
    1: 'Data 1', 2: 'Data 2', 3: 'Data 3', 4: 'Data 4', 5: 'Data 5', 
    6: 'Data 6', 7: 'Data 7', 8: 'Data 8', 9: 'Data 9', 10: 'Data 10',
}

In [48]:
def visualization1(resonant_frequency, max_peak_amplitude, amplitude_spectrum, label_index):   

    label_value = dict[label_index]
    
    # 创建图形和坐标轴
    fig, ax = plt.subplots()
    # 绘制频谱图
    ax.plot(freq[:n//2], amplitude_spectrum[:n//2], label= label_value)
    ax.plot(resonant_frequency, max_peak_amplitude, 'ro', label='Peak Point')
    
    ax.xaxis.set_tick_params(direction='in')
    ax.yaxis.set_tick_params(direction='in')

    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Amplitude Spectrum')
    ax.set_title('Fourier Transform', style='italic')

    ax.legend()
          
    # 创建保存文件的文件夹
    folder_name = 'result/fourier_transform'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # 保存图形到 result 文件夹
    file_name = f'{folder_name}/FFT_{label_value}.png'
    plt.savefig(file_name)
    plt.close() # 注释该行可使图像显示

In [49]:
def visualization2(result_peak, label_index):   

    label_value = dict[label_index]

    fig, ax = plt.subplots()
    # 绘制时程图
    ax.plot(time[:100], data.iloc[label_index][:100], label= label_value)
    ax.plot(result_peak.iloc[label_index-1, 1], result_peak.iloc[label_index-1, 2], 'ro', label='Peak Point 1')
    ax.plot(result_peak.iloc[label_index-1, 3], result_peak.iloc[label_index-1, 4], 'ro', label='Peak Point 2')

    ax.xaxis.set_tick_params(direction='in')
    ax.yaxis.set_tick_params(direction='in')

    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Displacement (mm)')
    ax.set_title('Time History Response', style='italic')

    ax.legend()
    
    # 创建保存文件的文件夹
    folder_name = 'result/time_history'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # 保存图形到 result 文件夹
    file_name = f'{folder_name}/TH_{label_value}.png'
    plt.savefig(file_name)
    plt.close()# 注释该行可使图像显示

In [50]:
def save_csv(file, file_name):
    result_folder = 'result'
    
    result_file_path = os.path.join(result_folder, file_name)
    file.to_csv(result_file_path, index=False)

In [58]:
def extract_peaks(data, time):
    # 找到波峰
    peaks, _ = signal.find_peaks(data, height=0)

    # 提取最初相邻两次波峰的具体值
    first_two_peaks = data.iloc[peaks[:2]]
    cor_time = time.iloc[peaks[:2]]
    delta_t= cor_time.iloc[1] - cor_time.iloc[0]
    frequency = 1 / delta_t
    damping_ratio = np.log(first_two_peaks.iloc[0] / first_two_peaks.iloc[1]) / (2 * np.pi)
    
    return first_two_peaks, cor_time, frequency, damping_ratio

In [63]:
result_df = pd.DataFrame(columns=['Label', 'Resonant Frequency'])  # 记录结构自振频率
result_peak = pd.DataFrame(columns=['Label', 't1', 'y1', 't2', 'y2', 'damping_ratio', 'frequency']) # 记录结构阻尼比

for i in range(1, 11):
    resonant_frequency, max_peak_amplitude, amplitude_spectrum = calculate(data.iloc[i])
    result_df = pd.concat([result_df, pd.DataFrame({'Label': [f'data{i}'], 'Resonant Frequency': [resonant_frequency]})], ignore_index=True)
    visualization1(resonant_frequency, max_peak_amplitude, amplitude_spectrum, i)

    peaks_data, peaks_time, frequency, damping_ratio = extract_peaks(data.iloc[i], data.iloc[0])
    result_peak = pd.concat([result_peak, pd.DataFrame({'Label': [f'data{i}'],'t1': [peaks_time.iloc[0]], 'y1': [peaks_data.iloc[0]],\
                                                        't2': [peaks_time.iloc[1]], 'y2': [peaks_data.iloc[1]],\
                                                        'damping_ratio': [damping_ratio], 'frequency': [frequency]})],\
                                                        ignore_index=True)
    visualization2(result_peak, i)
    

In [64]:
save_csv(result_df, 'Resonant_frequency_table.csv')
save_csv(result_peak, 'Damping_ratio_calculation_table.csv')